In [1]:
import pymongo
import datetime
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import numpy as np
import pandas as pd
import json
import kafka
import pymongo
import logging
from transformers import pipeline
from transformers import BartTokenizer, PegasusTokenizer
from transformers import T5Tokenizer, T5ForConditionalGeneration, T5Config
from transformers import BartForConditionalGeneration, PegasusForConditionalGeneration

import warnings
warnings.filterwarnings("ignore")

from tqdm import tqdm

d:\Vin_TaiLieu_Binh\NLP\project\env\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [9]:
def get_random_blog(year, num):
    myclient = pymongo.MongoClient("mongodb://localhost:27017/")
    mydb = myclient["app"]
    mycol = mydb["blog_raw"]
    mycol2 = mydb["clf_blog"]

    # get id from clf_twitter that predicted as 1
    id_list = [x['id'] for x in mycol2.find({'prediction': "1"}).limit(num)]

    # get random tweets from twitter_raw
    random_blog = []
    for id in id_list:
        random_blog.append(mycol.find_one({'id': id}))

    return random_blog

In [23]:
MAX_LENGTH = 512

model = BartForConditionalGeneration.from_pretrained('hoabinh/finetune')
tokenizer = BartTokenizer.from_pretrained('hoabinh/finetune')

client = pymongo.MongoClient('mongodb://localhost:27017/')
db = client['app']
collection = db['sum_blog2']

def summary(data):
    inputs = tokenizer([data['content']], max_length=MAX_LENGTH, return_tensors="pt", truncation=True,padding="longest")
    summary_ids = model.generate(inputs['input_ids'], num_beams=2, min_length=30, max_length=128)
    summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)

    inputs = tokenizer([data['content']], max_length=512, truncation=True,
                            padding="max_length", return_tensors="pt")
        
    summaries = model.generate(input_ids=inputs["input_ids"],
                                attention_mask=inputs["attention_mask"],
                                length_penalty=0.8, num_beams=8, max_length=512)
        
    decoded_summaries = tokenizer.decode(summaries[0], skip_special_tokens=True,
                            clean_up_tokenization_spaces=True)
    
    # decoded_summaries = [d.replace("<n>", " ") for d in decoded_summaries]


    record  = {
        'id': data['id'],
        'summary': decoded_summaries,
    }
    
    collection.insert_one(record)


In [25]:
random_blog = get_random_blog(2022, 100)
record_content = [x['content'] for x in random_blog]

In [26]:
for data in tqdm(random_blog):
    summary(data)

 59%|█████▉    | 59/100 [22:31<15:07, 22.13s/it]

In [3]:
myclient = pymongo.MongoClient("mongodb://localhost:27017/")
mydb = myclient["app"]
mycol = mydb["blog_raw"]
mycol2 = mydb["sum_blog2"]

id_sum = [x['id'] for x in mycol2.find()]

lid = []
ltitle = []
lcontent = []
lsummary = []
ldate = []

for id in id_sum:
    # get data from blog_raw by id
    data = mycol.find_one({'id': id})
    # get summary from sum_blog by id
    summary = mycol2.find_one({'id': id})
    # print('id: ', id)
    # print('content: ', data['content'])
    # print('summary: ', summary['summary'])
    lid.append(id)
    ltitle.append(data['title'])
    # lcontent.append(data['content'])
    lsummary.append(summary['summary'])
    ldate.append(data['date'])

df = pd.DataFrame({'id': lid, 'date': ldate, 'title': ltitle, 'summary': lsummary})
df['summary'] = df['summary'].apply(lambda x: x[0].upper() + x[1:])
df

,id,date,title,summary
0,fa4f5919-e419-53e3-a157-f358b7a49836,19-September-2022,Can your iPhone be hacked? What to know about ...,"In this article, we’ll look at some of the mos..."
1,1f49d0be-00b4-5916-9f33-01a6c10c8c78,28-September-2022,Who Could Be Behind the Latest GitHub-Hosted M...,WhoisXML API threat researcher Dancho Danchev...
2,d4796dd3-c99d-52aa-850a-5236310acbde,22-September-2022,"Hey WeLiveSecurity, how does biometric authent...",As biometric identification becomes more commo...
3,702a9fc0-6bfa-5456-884f-6b77d2a0507b,11-June-2021,About the Unsuccessful Quest for a Deserializa...,This blog post describes the research on SAP J...
4,f2ffe625-54db-582b-a8fa-13b2d248bb02,04-October-2022,Probing an Active Digital Trail of Iranian Hac...,WhoisXML API threat researcher Dancho Danchev...
5,21784986-aad2-5a58-bad2-7615dc1a5a4c,19-July-2022,Under Siege for Months: the Anatomy of an Indu...,The Cyber-Threat Intelligence Lab at Bitdefend...
6,0dca21fc-f7bb-5f47-81ce-879089ddb1d7,21-September-2016,Once Upon a Type Confusion,Microsoft released the MS16–107 to patch CVE-2...
7,c5a18475-cd79-5edd-8b53-8de1c71f6639,25-January-2019,How Does The SDK Version Affect The Security o...,We were recently tasked with determining what ...
8,0348018c-4add-5e65-a3d9-6733f130066e,17-August-2022,Real-Time Behavior-Based Detection on Android ...,"For the past few years, cyber-criminals have s..."
9,7bb34cb4-8cec-5712-9b76-46b4a70b04a7,21-September-2021,RCE in Citrix ShareFile Storage Zones Controll...,"In April, Citrix published an advisory that ad..."


In [4]:
df.to_csv('data/blog_summary_v2.csv', index=False)

In [13]:
import pandas as pd 

test = pd.read_csv('data/blog_summary_v2.csv')
test

,id,date,title,summary
0,fa4f5919-e419-53e3-a157-f358b7a49836,19-September-2022,Can your iPhone be hacked? What to know about ...,"In this article, we’ll look at some of the mos..."
1,1f49d0be-00b4-5916-9f33-01a6c10c8c78,28-September-2022,Who Could Be Behind the Latest GitHub-Hosted M...,WhoisXML API threat researcher Dancho Danchev...
2,d4796dd3-c99d-52aa-850a-5236310acbde,22-September-2022,"Hey WeLiveSecurity, how does biometric authent...",As biometric identification becomes more commo...
3,702a9fc0-6bfa-5456-884f-6b77d2a0507b,11-June-2021,About the Unsuccessful Quest for a Deserializa...,This blog post describes the research on SAP J...
4,f2ffe625-54db-582b-a8fa-13b2d248bb02,04-October-2022,Probing an Active Digital Trail of Iranian Hac...,WhoisXML API threat researcher Dancho Danchev...
5,21784986-aad2-5a58-bad2-7615dc1a5a4c,19-July-2022,Under Siege for Months: the Anatomy of an Indu...,The Cyber-Threat Intelligence Lab at Bitdefend...
6,0dca21fc-f7bb-5f47-81ce-879089ddb1d7,21-September-2016,Once Upon a Type Confusion,Microsoft released the MS16–107 to patch CVE-2...
7,c5a18475-cd79-5edd-8b53-8de1c71f6639,25-January-2019,How Does The SDK Version Affect The Security o...,We were recently tasked with determining what ...
8,0348018c-4add-5e65-a3d9-6733f130066e,17-August-2022,Real-Time Behavior-Based Detection on Android ...,"For the past few years, cyber-criminals have s..."
9,7bb34cb4-8cec-5712-9b76-46b4a70b04a7,21-September-2021,RCE in Citrix ShareFile Storage Zones Controll...,"In April, Citrix published an advisory that ad..."


In [14]:
from datetime import datetime

date = []
for d in test['date']:
    try:
        date.append(datetime.strptime(d, '%d-%B-%Y').strftime('%Y-%m-%d'))
    except:
        date.append('')
    #     continue
    # date.append(datetime.strptime(d, '%Y-%m-%d'))


# d = [datetime.strptime(s, '%d-%B-%Y').strftime('%Y-%m-%d') for s in test.date]
test.date = date
# text sort by date
test = test.sort_values(by=['date'], ascending=False)
test

,id,date,title,summary
41,c28e73f2-fd1c-52e8-b316-f1c577a4f299,2022-10-20,Medibank confirms hackers have customer data,"“That data includes first names and surnames,..."
30,5399287c-ccec-595f-8d4c-3310546d21ef,2022-10-18,The benefits of taking an intent-based approac...,Cisco Talos is taking a different approach and...
27,6fc80ebc-7bbe-57ec-91dd-f4612ca4afe1,2022-10-11,On the Frontlines of the Syrian Electronic Arm...,WhoisXML API threat researcher Dancho Danchev...
16,1d57ea56-1b58-5f46-9b86-085d865ab092,2022-10-07,The Inner Workings of the Russian Business Net...,The Russian Business Network (RBN) Internet Se...
36,32193e11-1230-5df7-b32d-8a01c41825f4,2022-10-05,Side-Loading OneDrive for profit – Cryptojacki...,"In this paper, we describe a cryptojacking cam..."
4,f2ffe625-54db-582b-a8fa-13b2d248bb02,2022-10-04,Probing an Active Digital Trail of Iranian Hac...,WhoisXML API threat researcher Dancho Danchev...
50,86e840e0-c115-5a3b-8c28-096050bee5d3,2022-10-04,Developer account body snatchers pose risks to...,Why are software supply chain attacks so effe...
53,497e0b1a-8959-52e0-96b4-727d240cd1ba,2022-09-30,Amazon‑themed campaigns of Lazarus in the Neth...,Lazarus APT group is targeting targets in the ...
49,0c2625bb-0252-57fd-a274-a06ded9038dc,2022-09-29,Brazilian Prilex Hackers Resurfaced With Sophi...,A Brazilian threat actor known as Prilex has r...
1,1f49d0be-00b4-5916-9f33-01a6c10c8c78,2022-09-28,Who Could Be Behind the Latest GitHub-Hosted M...,WhoisXML API threat researcher Dancho Danchev...


In [17]:
test = test.dropna()
test = test.reset_index(drop=True)
test = test[:55]
test

,id,date,title,summary
0,c28e73f2-fd1c-52e8-b316-f1c577a4f299,2022-10-20,Medibank confirms hackers have customer data,"“That data includes first names and surnames,..."
1,5399287c-ccec-595f-8d4c-3310546d21ef,2022-10-18,The benefits of taking an intent-based approac...,Cisco Talos is taking a different approach and...
2,6fc80ebc-7bbe-57ec-91dd-f4612ca4afe1,2022-10-11,On the Frontlines of the Syrian Electronic Arm...,WhoisXML API threat researcher Dancho Danchev...
3,1d57ea56-1b58-5f46-9b86-085d865ab092,2022-10-07,The Inner Workings of the Russian Business Net...,The Russian Business Network (RBN) Internet Se...
4,32193e11-1230-5df7-b32d-8a01c41825f4,2022-10-05,Side-Loading OneDrive for profit – Cryptojacki...,"In this paper, we describe a cryptojacking cam..."
5,f2ffe625-54db-582b-a8fa-13b2d248bb02,2022-10-04,Probing an Active Digital Trail of Iranian Hac...,WhoisXML API threat researcher Dancho Danchev...
6,86e840e0-c115-5a3b-8c28-096050bee5d3,2022-10-04,Developer account body snatchers pose risks to...,Why are software supply chain attacks so effe...
7,497e0b1a-8959-52e0-96b4-727d240cd1ba,2022-09-30,Amazon‑themed campaigns of Lazarus in the Neth...,Lazarus APT group is targeting targets in the ...
8,0c2625bb-0252-57fd-a274-a06ded9038dc,2022-09-29,Brazilian Prilex Hackers Resurfaced With Sophi...,A Brazilian threat actor known as Prilex has r...
9,1f49d0be-00b4-5916-9f33-01a6c10c8c78,2022-09-28,Who Could Be Behind the Latest GitHub-Hosted M...,WhoisXML API threat researcher Dancho Danchev...


In [21]:
for i in range(len(test)):
    print(i)
    print(test['title'][i])
    print("")
    print(test['summary'][i])
    print("================")

0
Medibank confirms hackers have customer data

 “That data includes first names and surnames, addresses, dates of birth, Medicare numbers, policy numbers, phone numbers and some claims data," it said. “where a customer received medical services, and codes relating to their diagnosis and procedures.” “This morning we will commence making direct contact with the affected customers to inform them of this latest development,” the insurer said.
1
The benefits of taking an intent-based approach to detecting Business Email Compromise

Cisco Talos is taking a different approach and using an intent-based model to identify and block Business Email Compromise (BEC) Scans BEC messages, irrespective of whether a threat actor is impersonating a C-level executive or any employee in an organization. 
2
On the Frontlines of the Syrian Electronic Army’s Digital Arsenal

 WhoisXML API threat researcher Dancho Danchev took a deeper look inside the Syrian Electronic Army’s digital infrastructure, revealin

In [22]:
test.to_csv('data/blog_summary_v2.csv', index=False)

In [23]:
def get_all_blog_label1():
    myclient = pymongo.MongoClient("mongodb://localhost:27017/")
    mydb = myclient["app"]
    mycol = mydb["blog_raw"]
    mycol2 = mydb["clf_blog"]

    # get id from clf_twitter that predicted as 1
    id_list = [x['id'] for x in mycol2.find({'prediction': "1"})]

    # get random tweets from twitter_raw
    random_blog = []
    for id in id_list:
        random_blog.append(mycol.find_one({'id': id}))

    return random_blog

In [24]:
blog_label1 = get_all_blog_label1()

In [25]:
record_content = [x['content'] for x in blog_label1]

In [26]:
len(record_content)

4256

In [29]:
df_blog1 = pd.DataFrame(blog_label1)
df_blog1

,_id,spd,url,title,author,date,content,tags,id
0,635124a328e9d467ce42824c,welivesec,https://www.welivesecurity.com/2022/09/19/can-...,Can your iPhone be hacked? What to know about ...,André Lameiras,19-September-2022,Here are some of the most common ways that an ...,[Mobile Security],fa4f5919-e419-53e3-a157-f358b7a49836
1,635124a328e9d467ce42824d,circleid,https://circleid.com/posts/20220928-who-could-...,Who Could Be Behind the Latest GitHub-Hosted M...,WhoisXML API,28-September-2022,GitHub is a popular code repository used by al...,"[Cybercrime, Cybersecurity, DNS, Domain Names,...",1f49d0be-00b4-5916-9f33-01a6c10c8c78
2,635124a428e9d467ce42824e,welivesec,https://www.welivesecurity.com/2022/09/22/hey-...,"Hey WeLiveSecurity, how does biometric authent...",André Lameiras,22-September-2022,"Your eyes may be the window to your soul, but ...",[Privacy],d4796dd3-c99d-52aa-850a-5236310acbde
3,635124a428e9d467ce42824f,codewhite,https://codewhitesec.blogspot.com/2021/06/abou...,About the Unsuccessful Quest for a Deserializa...,Code White,11-June-2021,This blog post describes the research on SAP J...,[],702a9fc0-6bfa-5456-884f-6b77d2a0507b
4,635124a428e9d467ce428250,circleid,https://circleid.com/posts/20221004-probing-an...,Probing an Active Digital Trail of Iranian Hac...,WhoisXML API,04-October-2022,WhoisXML API threat researcher Dancho Danchev ...,"[Brand Protection, Cybercrime, Cybersecurity, ...",f2ffe625-54db-582b-a8fa-13b2d248bb02
...,...,...,...,...,...,...,...,...,...
4251,6354b32a5edf360a3703b898,infosec,https://resources.infosecinstitute.com/topic/c...,Cybersecurity Weekly: Police trick ransomware ...,Camille Raymond,17-October-2022,Police trick ransomware gang by faking ransom ...,[News],ff01deb3-8d19-5899-9016-bf2f467d6b83
4252,6354b3315edf360a3703b89c,itnews,https://www.itnews.com.au/news/uk-banks-tech-s...,"UK Banks, tech seek clearer guidance on combat...",Huw Jones,21-October-2022,"Banks, tech and telecom companies in Britain a...","[bank, data breach, finance, privacy, scams]",4988175b-6c64-53e5-9d1b-8f1adaa8b361
4253,6354b3485edf360a3703b8a0,hackernew,https://thehackernews.com/2022/10/emotet-botne...,Emotet Botnet Distributing Self-Unlocking Pass...,Ravie Lakshmanan,21-October-2022,The notorious Emotet botnet has been linked to...,[Malware],6cc1adf4-2e28-50e8-9c94-51201d282227
4254,6354b34a5edf360a3703b8a1,hackernew,https://thehackernews.com/2022/10/hackers-star...,"Hackers Started Exploiting Critical ""Text4Shel...",Ravie Lakshmanan,21-October-2022,WordPress security company Wordfence on Thursd...,[Malware],4e3b28fd-3904-5e7b-8d98-a2602ac5cbd5


In [30]:
from datetime import datetime

date = []
for d in df_blog1['date']:
    try:
        date.append(datetime.strptime(d, '%d-%B-%Y').strftime('%Y-%m-%d'))
    except:
        date.append('')
    #     continue
    # date.append(datetime.strptime(d, '%Y-%m-%d'))


# d = [datetime.strptime(s, '%d-%B-%Y').strftime('%Y-%m-%d') for s in test.date]
df_blog1.date = date
# text sort by date
df_blog1 = df_blog1.sort_values(by=['date'], ascending=False)
df_blog1

,_id,spd,url,title,author,date,content,tags,id
4255,6354b34c5edf360a3703b8a2,hackernew,https://thehackernews.com/2022/10/multiple-cam...,Multiple Campaigns Exploit VMware Vulnerabilit...,Ravie Lakshmanan,2022-10-21,A now-patched vulnerability in VMware Workspac...,[Malware],d3e7785c-b293-5aed-bb2b-056059318c45
4254,6354b34a5edf360a3703b8a1,hackernew,https://thehackernews.com/2022/10/hackers-star...,"Hackers Started Exploiting Critical ""Text4Shel...",Ravie Lakshmanan,2022-10-21,WordPress security company Wordfence on Thursd...,[Malware],4e3b28fd-3904-5e7b-8d98-a2602ac5cbd5
4253,6354b3485edf360a3703b8a0,hackernew,https://thehackernews.com/2022/10/emotet-botne...,Emotet Botnet Distributing Self-Unlocking Pass...,Ravie Lakshmanan,2022-10-21,The notorious Emotet botnet has been linked to...,[Malware],6cc1adf4-2e28-50e8-9c94-51201d282227
4252,6354b3315edf360a3703b89c,itnews,https://www.itnews.com.au/news/uk-banks-tech-s...,"UK Banks, tech seek clearer guidance on combat...",Huw Jones,2022-10-21,"Banks, tech and telecom companies in Britain a...","[bank, data breach, finance, privacy, scams]",4988175b-6c64-53e5-9d1b-8f1adaa8b361
4250,6354b3295edf360a3703b896,circleid,https://circleid.com/posts/20221021-alleviatin...,Alleviating BlackEnergy-Enabled DDoS Attacks,WhoisXML API,2022-10-21,BlackEnergy first appeared in 2007. Designed t...,"[Cybersecurity, DDoS Attack, DNS, Malware, Thr...",0fe37ddd-cac0-59f3-8eda-f9740168e5c3
...,...,...,...,...,...,...,...,...,...
214,635124be28e9d467ce428342,telsy,https://www.telsy.com/google-drive-abused-in-d...,Google Drive abused in document exfiltration o...,Telsy,,Telsy analyzed a new campaign that targeted Af...,"[Cybersecurity, Cyber Threat Intelligence, Cyb...",ffef0914-be14-5360-a4e7-54ad7d71d1f4
372,635124d028e9d467ce4283ff,telsy,https://www.telsy.com/the-lazarus-gaze-to-the-...,The Lazarus’ gaze to the world: What is behind...,Telsy,,In a recent blog post ( link here ) we analyse...,[Cyber Threat Intelligence],052781b8-e06e-5dc5-af25-f0a91a9d8d5c
203,635124bd28e9d467ce428336,telsy,https://www.telsy.com/remcos-and-agent-tesla-l...,REMCOS and Agent Tesla loaded into memory with...,Telsy,,An Italian government email has been targeted ...,"[Cybersecurity, Cyber Threat Intelligence, Cyb...",f4d62db0-4618-5121-bf3e-898354e33d2c
379,635124d028e9d467ce428407,mcafee,https://www.mcafee.com/blogs/privacy-identity-...,9 Ways to Determine If Your Identity Has Been ...,McAfee,,"Most of us use the internet every day, so we’r...",[None],ec2eea65-075c-5cc6-a7f0-08464fcd40fb


In [32]:
df_blog1 = df_blog1[:1000]
df_blog1 = df_blog1.dropna().reset_index(drop=True)
df_blog1

,_id,spd,url,title,author,date,content,tags,id
0,6354b34c5edf360a3703b8a2,hackernew,https://thehackernews.com/2022/10/multiple-cam...,Multiple Campaigns Exploit VMware Vulnerabilit...,Ravie Lakshmanan,2022-10-21,A now-patched vulnerability in VMware Workspac...,[Malware],d3e7785c-b293-5aed-bb2b-056059318c45
1,6354b34a5edf360a3703b8a1,hackernew,https://thehackernews.com/2022/10/hackers-star...,"Hackers Started Exploiting Critical ""Text4Shel...",Ravie Lakshmanan,2022-10-21,WordPress security company Wordfence on Thursd...,[Malware],4e3b28fd-3904-5e7b-8d98-a2602ac5cbd5
2,6354b3485edf360a3703b8a0,hackernew,https://thehackernews.com/2022/10/emotet-botne...,Emotet Botnet Distributing Self-Unlocking Pass...,Ravie Lakshmanan,2022-10-21,The notorious Emotet botnet has been linked to...,[Malware],6cc1adf4-2e28-50e8-9c94-51201d282227
3,6354b3315edf360a3703b89c,itnews,https://www.itnews.com.au/news/uk-banks-tech-s...,"UK Banks, tech seek clearer guidance on combat...",Huw Jones,2022-10-21,"Banks, tech and telecom companies in Britain a...","[bank, data breach, finance, privacy, scams]",4988175b-6c64-53e5-9d1b-8f1adaa8b361
4,6354b3295edf360a3703b896,circleid,https://circleid.com/posts/20221021-alleviatin...,Alleviating BlackEnergy-Enabled DDoS Attacks,WhoisXML API,2022-10-21,BlackEnergy first appeared in 2007. Designed t...,"[Cybersecurity, DDoS Attack, DNS, Malware, Thr...",0fe37ddd-cac0-59f3-8eda-f9740168e5c3
...,...,...,...,...,...,...,...,...,...
995,6351290a967479052df71eb0,microsoft,https://www.microsoft.com/security/blog/2021/0...,FoggyWeb: Targeted NOBELIUM malware leads to p...,Ramin Nafisi,2021-09-27,Microsoft continues to work with partners and ...,[Cybersecurity],9c80d228-99b6-5160-b781-395006986f99
996,63518ef0a0885f14a17295a9,ciscotalos,https://blog.talosintelligence.com/2021/09/thr...,Threat Roundup for September 17 to September 24,William Largent,2021-09-24,"Today, Talos is publishing a glimpse into the ...",[Threat Roundup],d26c0dc9-90b0-5683-b47a-c8523e6f5466
997,63512954967479052df72242,secura,https://www.secura.com/whitepapers/otcad,OTCAD: Operational Technology Cyber Attack Dat...,Secura,2021-09-24,"At its release, OTCAD contains data of 133 pub...","[White papers, Energy / Utilities]",9a183e25-27d1-5433-aef6-583bda582f69
998,63518eeca0885f14a1729595,ciscotalos,https://blog.talosintelligence.com/2021/09/ope...,Operation “Armor Piercer:” Targeted attacks in...,Asheer Malhotra,2021-09-23,"By Asheer Malhotra , Vanja Svajcer and Justin ...","[RATs, SecureX, spam, threats]",55402f62-a632-5f58-9d0a-d2c1f40dcf10


In [33]:
df_blog1.to_csv('data/blog_label1_sortbydate.csv', index=False)

In [34]:
import torch

torch.cuda.is_available()

False